## Linear Elasticity for Cantilever Beam

In [29]:
using Gridap, Gridap.CellData, SparseArrays, BenchmarkTools
using NearestNeighbors, Printf  # make sure this is at the top of your file
using Gridap
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using InteractiveUtils
using Gridap.Polynomials
using LinearAlgebra


## Inputs

### Add labels to left and right faces for boundary conditions

In [ ]:

L = 20 # Length
W = 10 # Width and Height
domain = (0, L, 0, W, 0, W) # sizes along X, Y, Z
partition = (20, 10, 10) # mesh sizes
model = CartesianDiscreteModel(domain, partition);
# Tag the left face (x = 0); right face (x = L)
labels = get_face_labeling(model);
add_tag_from_tags!(labels,"left", [1,3,5,7,13,15,17,19,25]); # left face: corners (1,3,5,7); edges (13,15,17,19); others (25) 
add_tag_from_tags!(labels, "right", [2,4,6,8,14,16,18,20,26]); # Right face: corners (2,4,6,8); edges (14,16,18,20); others (26) 
# writevtk(Ω,"Cantilever_Beam_3D")
order = 1
reffe = LagrangianRefFE(VectorValue{3,Float64}, HEX, order)
# Integrating on the domain Ω
degree = 2
Ωₕ = Triangulation(model)
dΩₕ = Measure(Ωₕ, degree)
# neumann boundaries
Γ = BoundaryTriangulation(model,tags = "right")
dΓ = Measure(Γ,degree)
# Vector-valued FE space

Vₕ = TestFESpace(Ωₕ, reffe; conformity=:H1, dirichlet_tags = "left")
Uₕ = TrialFESpace(Vₕ)

Qₕ = CellQuadrature(Ωₕ,degree)

ν = 0.3   # Poisson's ratio
E = 119e3 # Young's modulus
# Lamé parameters
μ = E / (2 * (1 + ν))
λ = ν * E / ((1 + ν) * (1 - 2 * ν))

# Constitutive matrix for 3D isotropic linear elasticity
# Lamé parameters
μ = E / (2 * (1 + ν))
λ = ν * E / ((1 + ν) * (1 - 2 * ν))

# Constitutive tensor for 3D isotropic linear elasticity
Ce(ε) = λ * tr(ε) * one(ε) + 2 * μ * ε

Qₕ_cell_data = get_data(Qₕ)
Qₕ_cell_point = get_cell_points(Qₕ)

dv = get_fe_basis(Vₕ)
du = get_trial_fe_basis(Uₕ)

Bilinear_form = evaluate(∇(du)  Ce(∇(du)), Qₕ_cell_point)



MethodError: MethodError: no method matching one(::Gridap.FESpaces.SingleFieldFEBasis{Gridap.FESpaces.TrialBasis, ReferenceDomain})

Closest candidates are:
  one(!Matched::Type{Union{}}, Any...)
   @ Base number.jl:349
  one(!Matched::Type{Missing})
   @ Base missing.jl:104
  one(!Matched::BitMatrix)
   @ Base bitarray.jl:426
  ...


In [37]:

# nodes = Point{2,Float64}[(0,0),(0,1),(0,2),(1,0),(1,1),(1,2),(2,0),(2,1),(2,2)] 
nodes = Fields.Point{2,Float64}[(0, 0), (1, 0), (2, 0), (0, 1), (1, 1), (2, 1), (0, 2), (1, 2), (2, 2)] # Avoi Point width point in Makie.Point
conn = [[1, 2, 4, 5], [2, 3, 5, 6], [4, 5, 7, 8], [5, 6, 8, 9]]

# Reference shape functions 
filter(e, p) = true
ref_nodes = Point{2,Float64}[
    (0, 0), (1, 0), (0, 1), (1, 1)]

m = MonomialBasis{2}(Float64, 1, filter)
l = LagrangianDofBasis(Float64, ref_nodes)
change = inv(evaluate(l, m))
s = linear_combination(change, m)
q = Point{2,Float64}[(0.5, 0.5)]
w = [1.]

# Cell-wise shape functions 
ncells = length(conn)
cell_s = Fill(s, ncells)
# Cell-wise quadrature 
cell_q = Fill(q, ncells)
cell_w = Fill(w, ncells)
# Geometrical map and Jacobian 
cell_nodes = lazy_map(Broadcasting(Arrays.Reindex(nodes)), conn)

cell_φ = lazy_map(
    linear_combination, cell_nodes, cell_s)
cell_Jt = lazy_map(∇, cell_φ)
# Cell-wise shape function derivatives
cell_∇ref_s = lazy_map(Broadcasting(∇), cell_s)
cell_invJt = lazy_map(Operation(inv), cell_Jt)
cell_∇s = lazy_map( Broadcasting(Operation(⋅)), cell_invJt, cell_∇ref_s)
#Cell-wise stifness matrix 50 cell_∇st = lazy_map(transpose,cell_∇s) 51 cell_∇s∇st = lazy_map( 52
 cell_∇st = lazy_map(transpose, cell_∇s)
cell_∇s∇st = lazy_map(  Broadcasting(Operation(⋅)), cell_∇s, cell_∇st)
cell_mat = lazy_map( integrate, cell_∇s∇st, cell_q, cell_w, cell_Jt)
cell_mat[1]

4×4 Matrix{Float64}:
  0.5   0.0   0.0  -0.5
  0.0   0.5  -0.5   0.0
  0.0  -0.5   0.5   0.0
 -0.5   0.0   0.0   0.5

## Run finite element analysis

In [6]:
# The weak form
F(x) = VectorValue(0.0, -1, 0.0) # A unit load along Y axis applies to each node on the right face
a(u,v) = ∫((σ∘ε₀(u)) ⊙ ε₀(v)) *dΩ # Left-hand size; (∘) Composite functions
l(v) = ∫(F ⋅ v) * dΓ # Right-hand size
# Solution of the FE problem
op = AffineFEOperator(a,l,U,V)
uh = solve(op)
# writevtk(Ω,"Cantilever_Beam_3D_uh",cellfields=["uh"=>uh,"epsi"=>ε₀(uh),"sigma"=>σ∘ε₀(uh)])

SingleFieldFEFunction():
 num_cells: 2000
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 12595192635354261362

### Compute the compliance

In [7]:
# Method 1
C1 = sum(∫((σ ∘ ε₀(uh)) ⊙ ε₀(uh)) * dΩ) # Compliance
println("Compliance 1: ", C1)
# Method 2
C2 = sum(∫(F ⋅ uh) * dΓ) # Compliance
println("Compliance 2: ", C2)
# Method 3
U = get_free_dof_values(uh) # displacement vector
K = get_matrix(op) # stiffness matrix
C3 = dot(U, K * U)
println("Compliance 3: ", C3)
# Method 4 using for loop to compute sum(ue^T*ke*ue)

using Gridap.Arrays, Gridap.FESpaces

cell_dofs_values = get_cell_dof_values(uh)
u = get_trial_fe_basis(V)
v = get_fe_basis(V)
K_e = get_array(∫((σ ∘ ε₀(u)) ⊙ ε₀(v)) * dΩ)
# ==

a1 = ∫((σ ∘ ε₀(u)) ⊙ ε₀(v)) * dΩ
KK = a1[Ω]
KK[1] .- K_e[1]
assem = SparseMatrixAssembler(V,V)
# Allocation global parametric stiffness
cell_dofs = get_cell_dof_ids(V)
data = ([KK],[cell_dofs],[cell_dofs])
KK_Global = allocate_matrix(assem,data)
# =========
numele = num_cells(Ω) # Number of elements
C4 = 0.0 # Comliance
for e=1:numele
    ue = cell_dofs_values[e]    # element displacement
    # ke = KK[e] # element stiffness matrix
    ke = K_e[e] # element stiffness matrix
    C4 = C4 + transpose(ue)*ke*ue
end
println("Compliance 4: ", C4)


Compliance 1: 0.30855010386860376
Compliance 2: 0.30855010386853077
Compliance 3: 0.30855010386865317
Compliance 4: 0.30855010386887693


In [11]:
u = get_trial_fe_basis(V)
u

SingleFieldFEBasis():
 num_cells: 2000
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 12595192635354261362

In [8]:
using Gridap
L = 100 # Length
W = 20 # Width and Height
domain = (0, L, 0, W, 0, W) # sizes along X, Y, Z
partition = (100, 20, 20) # mesh sizes
model = CartesianDiscreteModel(domain, partition)
# Tag the left face (x = 0); right face (x = L)
labels = get_face_labeling(model)
add_tag_from_tags!(labels,"left", [1,3,5,7,13,15,17,19,25]) # left face: corners (1,3,5,7); edges (13,15,17,19); others (25) 
add_tag_from_tags!(labels, "right", [2,4,6,8,14,16,18,20,26]) # Right face: corners (2,4,6,8); edges (14,16,18,20); others (26) 
# Integrating on the domain Ω
degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω,degree)
# neumann boundaries
Γ = BoundaryTriangulation(model,tags = "right")
dΓ = Measure(Γ,degree)
# Vector-valued FE space
order = 1
reffe = ReferenceFE(lagrangian, VectorValue{3,Float64}, order)
Vₕ = TestFESpace(Ω, reffe; conformity=:H1, dirichlet_tags = "left")
Uₕ = TrialFESpace(Vₕ)
# Constitutive law for LINEAR ELASTICITY 
const E = 119e3 # Young's modulus
const ν = 0.3   # Poisson's ratio
const λ = (E*ν)/((1+ν)*(1-2*ν)) # Lamé parameters
const μ = E/(2*(1+ν)) # Lamé parameters
ε₀(u) = 0.5 * ( ∇(u) + transpose(∇(u))) # Strain; In Gridap this can be automatically defined in Gridap.ε
σ(ε₀) = λ * tr(ε₀) * one(ε₀) + 2μ * ε₀ # Stress
# The weak form
F(x) = VectorValue(0.0, -1, 0.0) # I want to apply a unit load along Y axis to each node on the right face
a(u,v) = ∫((σ∘ε₀(u)) ⊙ ε₀(v)) *dΩ # Left-hand size; (∘) Composite functions
l(v) = ∫(F ⋅ v) * dΓ # Right-hand size
# Solution of the FE problem
op = AffineFEOperator(a,l,Uₕ,Vₕ)
uh = solve(op)
# Export the displacement, strain, stress



SingleFieldFEFunction():
 num_cells: 40000
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 648537071230102027

In [9]:
Qₕ = CellQuadrature(Ω,4*order)
Qₕ_cell_point = get_cell_points(Qₕ)
Qₕ_cell_data = get_data(Qₕ)
dv = get_fe_basis(Vₕ)
du = get_trial_fe_basis(Uₕ)
grad_dv = ∇(dv)
grad_dv_array = get_data(grad_dv)
using Gridap.Fields

aa=evaluate(grad_dv,Fields.Point{3,Float64}[(0.5, 0.5, 0.5)])


1-element Vector{TensorValue{3, 3, Float64, 9}}:
 TensorValue{3, 3, Float64, 9}(-0.25, -0.25, -0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)